# Credit Risk Resampling Techniques

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [62]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
import matplotlib.pyplot as plt

In [3]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Read the CSV and Perform Basic Data Cleaning

In [4]:
columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [5]:
#file_path = Path('LoanStats_2019Q1.csv.zip')
#df = pd.read_csv(file_path, skiprows=1)[:-2]
#df = df.loc[:, columns].copy()


In [6]:
# Load the data
file_path = Path('LoanStats_2019Q1.csv.zip')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

len(df)

68817

In [7]:
df = pd.read_csv("11-Machine_Learning")
df.tail()

,loan_amnt,int_rate,installment,annual_inc,loan_status,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
68812,10000.0,0.1502,346.76,26000.0,1,9.60,0.0,0.0,9.0,0.0,...,0.0,3.0,80.0,0.0,0.0,0.0,20625.0,6798.0,11300.0,5425.0
68813,12000.0,0.2727,368.37,63000.0,1,29.07,0.0,0.0,8.0,0.0,...,0.0,5.0,96.2,0.0,0.0,0.0,87939.0,60350.0,13500.0,62939.0
68814,5000.0,0.1992,185.62,52000.0,1,14.86,0.0,0.0,5.0,1.0,...,0.0,3.0,100.0,0.0,1.0,0.0,30592.0,18611.0,3600.0,18492.0
68815,40000.0,0.0646,1225.24,520000.0,1,9.96,0.0,1.0,21.0,0.0,...,0.0,3.0,98.2,12.5,0.0,0.0,1033574.0,95958.0,100800.0,78634.0
68816,16000.0,0.1131,350.36,72000.0,1,7.02,2.0,0.0,12.0,1.0,...,2.0,0.0,94.3,0.0,1.0,0.0,251486.0,74835.0,23000.0,63090.0


In [8]:
len(df)

68817

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68817 entries, 0 to 68816
Data columns (total 77 columns):
loan_amnt                     68817 non-null float64
int_rate                      68817 non-null float64
installment                   68817 non-null float64
annual_inc                    68817 non-null float64
loan_status                   68817 non-null int64
dti                           68817 non-null float64
delinq_2yrs                   68817 non-null float64
inq_last_6mths                68817 non-null float64
open_acc                      68817 non-null float64
pub_rec                       68817 non-null float64
revol_bal                     68817 non-null float64
total_acc                     68817 non-null float64
out_prncp                     68817 non-null float64
out_prncp_inv                 68817 non-null float64
total_pymnt                   68817 non-null float64
total_pymnt_inv               68817 non-null float64
total_rec_prncp               68817 non-null 

# Split the Data into Training and Testing

In [10]:
# Create our features
X =  df.drop(columns="loan_status")

# Create our target
y = df["loan_status"]

In [11]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,6.881700e+04,6.881700e+04,68817.000000,6.881700e+04
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,0.052138,2.219423,95.057627,30.626217,0.125972,0.0,2.100332e+05,6.133843e+04,29734.128558,5.572240e+04
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.390633,1.897432,8.326426,33.631463,0.336732,0.0,1.928088e+05,5.738798e+04,26795.394232,5.095845e+04
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,0.000000,20.000000,0.000000,0.000000,0.0,3.600000e+03,2.350000e+02,100.000000,1.270000e+02
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,0.000000,1.000000,93.000000,0.000000,0.000000,0.0,6.697700e+04,2.650300e+04,11600.000000,2.288000e+04
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,0.000000,2.000000,100.000000,20.000000,0.000000,0.0,1.467100e+05,4.535700e+04,22100.000000,4.200000e+04
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,0.000000,3.000000,100.000000,50.000000,0.000000,0.0,3.036400e+05,7.657000e+04,39300.000000,7.249900e+04
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,18.000000,19.000000,100.000000,100.000000,4.000000,0.0,3.292782e+06,1.295455e+06,509400.000000,1.426964e+06


In [12]:
y.value_counts()

1    68470
0      347
Name: loan_status, dtype: int64

In [13]:
# Check the balance of our target values
y.value_counts()

1    68470
0      347
Name: loan_status, dtype: int64

In [14]:
# Create X_train, X_test, y_train, y_test
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=1)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [15]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [16]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_train_scaler = scaler.fit(X_train)
X_train_scaled = X_train_scaler.transform(X_train)
X_train_scaled[:2]

array([[-1.46833559, -0.82153497, -1.49317685, -0.42586799,  0.12589882,
        -0.30522104, -0.65473938,  1.23295488, -0.37456588, -0.75167716,
         0.27246219, -1.44662473, -1.44600927, -0.76716958, -0.76699171,
        -0.56534335, -1.07769486, -0.03351112,  0.        ,  0.        ,
        -0.59376056, -0.12149415,  0.        ,  0.        , -0.10241123,
        -0.57875387, -0.86782359, -0.77085757,  0.16663114,  0.06517785,
        -0.52435836, -0.60788751,  0.84729764, -0.84533094,  0.15413312,
        -1.06996889, -2.14652184,  0.81979116,  0.40528984, -0.58955494,
        -0.49420782,  0.37972927, -0.69677319,  1.98439746, -1.68036353,
        -0.07969493, -0.00899006, -1.10196322, -1.16958871,  0.44872516,
        -0.03201589, -0.25186321, -0.08520337, -0.11734207, -0.33784239,
         0.07838573, -0.18640096,  2.43041976,  1.65679501, -0.72654543,
         1.86016815,  1.18584178, -0.1877791 ,  1.23570218,  0.        ,
         0.        , -0.13469636, -0.6420923 ,  0.5

In [17]:
# Scale the training and testing data
# YOUR CODE HERE

# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [18]:
# Resample the training data with the RandomOversampler
# YOUR CODE HERE

In [19]:
# Train the Logistic Regression model using the resampled data
# YOUR CODE HERE

In [20]:
# Calculated the balanced accuracy score
# YOUR CODE HERE

In [21]:
# Display the confusion matrix
# YOUR CODE HERE

In [22]:
# Print the imbalanced classification report
# YOUR CODE HERE

### SMOTE Oversampling

In [23]:
from imblearn.over_sampling import SMOTE

In [24]:
from collections import Counter

In [25]:
X_smote,y_smote = SMOTE(random_state=1).fit_resample(X_train,y_train)

In [26]:
Counter(y_smote)

Counter({1: 51366, 0: 51366})

In [27]:
# Resample the training data with SMOTE
# YOUR CODE HERE

In [28]:
from sklearn.linear_model import LogisticRegression

In [29]:
lr = LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100,
          n_jobs=None, penalty='l2', random_state=1, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)

In [30]:
lr.fit(X_smote,y_smote)

LogisticRegression(random_state=1)

In [31]:
# Train the Logistic Regression model using the resampled data
# YOUR CODE HERE

In [32]:
from sklearn.metrics import confusion_matrix

In [33]:
lr_predictions = lr.predict(X_test)

In [34]:
cm = confusion_matrix(y_test,lr_predictions)
cm

array([[   66,    35],
       [ 5523, 11581]], dtype=int64)

In [35]:
# Display the confusion matrix
# YOUR CODE HERE

In [36]:
from sklearn.metrics import balanced_accuracy_score

In [37]:
bal_score = balanced_accuracy_score(y_test,lr_predictions)
bal_score

0.665279212088655

In [38]:
# Calculated the balanced accuracy score
# YOUR CODE HERE

In [39]:
from imblearn.metrics import classification_report_imbalanced

In [40]:
imbalanced = classification_report_imbalanced(y_test,lr_predictions)
print(imbalanced)

                   pre       rec       spe        f1       geo       iba       sup

          0       0.01      0.65      0.68      0.02      0.67      0.44       101
          1       1.00      0.68      0.65      0.81      0.67      0.44     17104

avg / total       0.99      0.68      0.65      0.80      0.67      0.44     17205



In [41]:
# Print the imbalanced classification report
# YOUR CODE HERE

# Undersampling

In this section, you will test an undersampling algorithms to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [42]:
from imblearn.under_sampling import RandomUnderSampler

In [43]:
RUS = RandomUnderSampler()

In [44]:
X_under,y_under = RUS.fit_resample(X_train,y_train)

In [45]:
Counter(y_train)

Counter({1: 51366, 0: 246})

In [46]:
# Resample the data using the ClusterCentroids resampler
# YOUR CODE HERE

In [47]:
lr.fit(X_under,y_under)

LogisticRegression(random_state=1)

In [48]:
under_prediction = lr.predict(X_test)

In [49]:
# Train the Logistic Regression model using the resampled data
# YOUR CODE HERE

In [50]:
bal_score = balanced_accuracy_score(y_test,under_prediction)

In [51]:
bal_score

0.6314960775778233

In [52]:
# Calculated the balanced accuracy score
# YOUR CODE HERE

In [53]:
cm2 = confusion_matrix(y_test,under_prediction)
cm2

array([[   67,    34],
       [ 6848, 10256]], dtype=int64)

In [54]:
# Display the confusion matrix
# YOUR CODE HERE

In [55]:
imbal = classification_report_imbalanced(y_test,under_prediction)
print(imbal)

                   pre       rec       spe        f1       geo       iba       sup

          0       0.01      0.66      0.60      0.02      0.63      0.40       101
          1       1.00      0.60      0.66      0.75      0.63      0.40     17104

avg / total       0.99      0.60      0.66      0.74      0.63      0.40     17205



In [56]:
# Print the imbalanced classification report
# YOUR CODE HERE

# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [57]:
from imblearn.combine import SMOTEENN

In [58]:
SMEE = SMOTEENN(random_state=1)

In [60]:
X_smoteenn,y_smoteenn = SMEE.fit_resample(X_train,y_train)

In [ ]:
# Resample the training data with SMOTEENN
# YOUR CODE HERE

In [72]:
lr = LogisticRegression(random_state=1)
lr.fit(X_smoteenn,y_smoteenn)
smoteenn_predictions = lr.predict(X_test)
smoteenn_predictions

array([1, 0, 1, ..., 0, 1, 1], dtype=int64)

In [ ]:
# Train the Logistic Regression model using the resampled data
# YOUR CODE HERE

In [76]:
smoteenn_bal_score = balanced_accuracy_score(y_test,smoteenn_predictions)
smoteenn_bal_score

0.6389999675832878

In [ ]:
# Calculated the balanced accuracy score
# YOUR CODE HERE

In [73]:
cm_smoteenn = confusion_matrix(y_test,smoteenn_predictions)
cm_smoteenn

array([[   69,    32],
       [ 6930, 10174]], dtype=int64)

In [ ]:
# Display the confusion matrix
# YOUR CODE HERE

In [74]:
smoteenn_imbal_score = classification_report_imbalanced(y_test,smoteenn_predictions)
print(smoteenn_imbal_score)

                   pre       rec       spe        f1       geo       iba       sup

          0       0.01      0.68      0.59      0.02      0.64      0.41       101
          1       1.00      0.59      0.68      0.75      0.64      0.40     17104

avg / total       0.99      0.60      0.68      0.74      0.64      0.40     17205



In [ ]:
# Print the imbalanced classification report
# YOUR CODE HERE